# **SALE WEB CRAWLING** #
### *Marlon A. Gutiérrez E.* ###

El presente código tiene como objetivo generar bases de datos de productos ofrecidos en el sitio web de almacenes Éxito, con ellas, encontrar productos que se encuentran con determinado descuento y efectuar un análisis de precios a lo largo del tiempo. Se elige almacenes Éxito por su frecuente cantidad de ofertas y altos descuentos, los cuales son de gran interés, especialmente, para mayoristas y clientes habituales. 

## **REQUERIMIENTOS:** ##

* Librerías instaladas: Matplotlib, Pandas, Random, Datetime, Selenium y time.

* Driver: en la misma ruta del presente archivo, se debe encontrar el archivo **geckodriver.exe**, el cual puede descargarse desde este link: https://github.com/mozilla/geckodriver/releases

* Aplicaciónes: Jupyter y Firefox.

## **Importación de módulos** ##

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import random
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep

## **Diccionario de páginas** ##

El sitio web que se está tratando organiza los productos en ciertas categorías principales, las cuales corresponden con una URL determinada. En este diccionario se agragan las URL de interés.

In [3]:
# Diccionario con key: nombre de categoría de productos y value: URL de cada categoría.
pages = {'Aseo hogar': 'https://www.exito.com/mercado/aseo-del-hogar',
         'Cuidado corporal': 'https://www.exito.com/salud-y-belleza/cuidado-corporal',
         'Despensa': 'https://www.exito.com/mercado/despensa'}

## **Función para obtener DataFrame de productos** ##

Esta función recibe una fecha de ejecución: "time [AñoMesDíaHoraMinuto]" (tipo str), una URL del diccionario anterior: "page" (tipo str) y un objeto: "driver" (tipo selenium webdriver). Entrega un DataFrame de productos de la respectiva URL.

El DataFrame que entrega consta de tres columnas con etiquetas 'HREF', 'PRODUCTO' y time=[AñoMesDíaHoraMinuto].

In [4]:
def get_df_products(time, page, driver):
    
    date = time    # Fecha de ejecución.
    
    url = page    # URL de la página de cada categoría de productos en el sitio web.
    
    driver.get(url)    # Con este método se carga el navegador Firefox con la respectiva URL.
    
    # Estas URL cargan 20 productos, para cargar más es necesario presionar el botón "Mostrar más".
    # Eso lo hace el siguiente ciclo.
    
    # '''
    
    i = True

    while i == True:
        
        # Hacer click sobre botones puede generar errores.
        try:
            
            # Encontrar el botón "Mostrar más".
            load_more = driver.find_element_by_xpath('//div[@class="vtex-button__label flex items-center justify-center h-100 ph5 "]')
            
            # Hacer click sobre el botón.
            load_more.click()
            
            # Esperar de 10 a 13 segundos para continuar el ciclo.
            sleep(random.uniform(10.0, 13.0))
            
            # La variable i continúa siendo True para continuar el ciclo.
            i = True
            
        except:
            
            # La variable i ahora será False para que salga del ciclo.
            i = False
            break
    
    # La cantidad de productos podría ser muy alta y se tomaría mucho tiempo cargando toda la página.
    # Para hacer pruebas es recomendable comentar el anterior ciclo while y descomentar el siguiente ciclo for para controlar la cantidad de cargas.
    
    '''
    
    for load in range(3):
        try:
            load_more = driver.find_element_by_xpath('//div[@class="vtex-button__label flex items-center justify-center h-100 ph5 "]')
            load_more.click()
            sleep(random.uniform(10.0, 13.0))
        except:
            break
            
    '''
    
    # Una vez cargada la página, se buscan los productos y se guardan en la variable products.

    products = driver.find_elements_by_class_name('vtex-search-result-3-x-galleryItem')
    
    # Se crea un DataFrame vacío con tres columnas.
    # 'HREF': referencia, 'PRODUCTO': nombre, date: precio.
    
    df_products = pd.DataFrame(columns=['HREF', 'PRODUCTO', date])
    
    # Se recorre el objeto products para obtener de él: href, nombre y precio de cada producto y almacenarlo en df_products.

    for item in products:
        
        # Obtener href del producto como referencia, ya que existen productos con el mismo nombre.
        a = item.find_element_by_xpath('.//a[@class="vtex-product-summary-2-x-clearLink h-100 flex flex-column"]')        
        href = a.get_attribute('href')
        href = href.lstrip('https://www.exito.com/')
        
        # Obtener nombre del producto.
        product_name = item.find_element_by_class_name('vtex-store-components-3-x-productNameContainer').text
        
        # Obtener precio del producto y dar formato para convertir en variable tipo int.
        product_price = item.find_element_by_class_name('exito-vtex-components-4-x-alliedDiscountPrice').text
        product_price = product_price.replace('.', '')
        product_price = product_price.lstrip('$ ')
        product_price = int(product_price)
        
        # Almacenar nombre y precio en columnas 'Producto' y 'date [AñoMesDíaHoraMinuto]', respectivamente.
        df_products = df_products.append({'HREF': href, 'PRODUCTO': product_name, date: product_price}, ignore_index=True)
    
    return df_products

## **Creación de archivos con valores iniciales** ##

Ejecutar la siguiente celda para crear un archivo xlsx para cada categoría de productos (*kind_ini*) con su DataFrame correspondiente.

* #### **Advertencia:** se reinician los archivos a datos actuales. Solo ejecutar cuando se desee iniciar el estudio de precios.  ####

In [42]:
# ADVERTENCIA: al ejecutar esta celda, se reiniciarán todos los archivos de precios.

# Fecha de ejecución.
date_ini = datetime.now().strftime('%Y%m%d%H%M')

# Se crea objeto para manipular el sitio web.
driver_ini = webdriver.Firefox()

# Se recorre el diccionario para acceder a cada categoría "kind_ini" y cada URL "url_ini".
for kind_ini, url_ini in pages.items():
    
    # Se crea DataFrame inicial por cada categoría de productos.
    df_ini = get_df_products(date_ini, url_ini, driver_ini)
    
    # Se genera archivo xlsx con cada DataFrame.
    df_ini.to_excel('Productos - {}.xlsx'.format(kind_ini), index=False)

## **Actualización de archivos** ##

Ejecutar la siguiente celda cada vez que se desee obtener los precios actuales de los productos para acumularlos como historial en las bases de datos (archivos xlsx).

In [4]:
# Fecha de ejecución.
date_new = datetime.now().strftime('%Y%m%d%H%M')

# Se crea objeto para manipular el sitio web.
driver_new = webdriver.Firefox()

# Se recorre el diccionario para acceder a cada categoría "kind_new" y cada URL "url_new".
for kind_new, url_new in pages.items():
    
    # Se crea DataFrame nuevo por cada categoría de productos.
    df_new = get_df_products(date_new, url_new, driver_new)
    
    # Se carga DataFrame antiguo (almacenado en disco) por cada archivo de productos.
    df_old = pd.read_excel('Productos - {}.xlsx'.format(kind_new))
    
    # Se mezclan los dos DataFrame acumulando los precios de cada actualización.
    df_fin = pd.merge(df_old, df_new, on=['HREF', 'PRODUCTO'], how='outer')
    
    # Se sobrescribe cada archivo con la nueva información.
    df_fin.to_excel('Productos - {}.xlsx'.format(kind_new), index=False)

## **Encontrar productos con descuento** ##

Ejecutar la siguiente celda para comparar los dos últimos precios registrados de cada producto y generar un archivo xlsx con los productos que cumplen determinado descuento.

In [4]:
# Fecha de ejecución.
date_sale = datetime.now().strftime('%Y%m%d%H%M')

# Descuento requerido. 
discount = 0.2

# Se recorren los archivos para buscar el descuento requerido.
for key in pages.keys():
    
    df = pd.read_excel('Productos - {}.xlsx'.format(key))
    df_sale = df[df.iloc[:, -1] < (1.0-discount)*df.iloc[:, -2]]
    
    # Si encuentra productos con descuento, genera un archivo agregándole a su nombre la fecha de ejecución.
    if len(df_sale.index) != 0:
        df_sale.to_excel('Productos - {} (ofertas {}).xlsx'.format(key, date_sale), index=False)

## **Búsqueda de productos por palabra clave** ##

Al ejecutar la siguiente celda se abre un input para ingresar el producto que se desea buscar. Luego se genera un DataFrame con los resultados y se imprime por pantalla.

In [ ]:
search = input('Buscar: ')

df_all = pd.DataFrame()

for key in pages.keys():
    df = pd.read_excel('Productos - {}.xlsx'.format(key))
    df_all = df_all.append(df, ignore_index=True)
    
df_search = df_all[df_all['PRODUCTO'].str.contains(search, case=False)]

# Opciones de impresión en pantalla del DataFrame.
pd.set_option('max_rows', None)
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

df_search

## **Gráfica de precios** ##

Después de realizar la búsqueda por palabra clave, puede determinarse la referencia HREF del producto e ingresarla en el input que genera la ejecución de la siguiente celda y así, obtener una gráfica del comportamiento del precio a lo largo del tiempo.

In [ ]:
search = input('Buscar: ')
print('\n')

df_all = pd.DataFrame()

for key in pages.keys():
    df = pd.read_excel('Productos - {}.xlsx'.format(key))
    df_all = df_all.append(df, ignore_index=True)

# DataFrame de única fila.
product_to_analyze = df_all[df_all['HREF']==search]

# Obetención del nombre del producto para establecer como título de la gráfica.
title = product_to_analyze.iloc[0, 1]

# Conversión a diccionario y eliminación de la columna 'HREF' y 'PRODUCTO'.
product_prices = product_to_analyze.to_dict('records')[0]
del product_prices['HREF']
del product_prices['PRODUCTO']

# Espacios a graficar.
x = list(product_prices.keys())
y = list(product_prices.values())

plt.plot(x, y)
plt.plot(x, y, 'ro')
plt.title(title, size=16)
plt.xlabel("Tiempo", size=14)
plt.ylabel("$ COP", size=14)
plt.grid()
plt.show()